# Workflow to create and maintain the validation table for CSLC Cal/Val

This notebook demonstrates how to query the public S3 bucket hosting granules associated with bursts identified for CSLC Cal/Val activities.

Specifically the motivation here to access + record the preqrequiste information from each granule in order to virtually perform Cal/Val analyses.

For reference, the following resources were used to help create this notebook:
https://alexwlchan.net/2017/listing-s3-keys/
https://github.com/boto/boto3/issues/1200

## Load prerequisite modules

In [1]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

import fsspec

import geopandas as gpd

import h5py

import pandas as pd

from pathlib import Path

import shapely.wkt as wkt

## Static variables that identify S3 paths to data

<div class="alert alert-warning">
Only change IF you know what you are doing (i.e. itentional changes to reflect hypothetical migration of validation data).
</div>

In [2]:
# set S3 path variables
bucket = 'opera-pst-rs-pop1'
prefix = 'products/CSLC_S1'
suffix = 'Z.h5'

s3_path = f's3://{bucket}'
DATA_ROOT = 'science/SENTINEL1'

# track specific version and corresponding static layer
# version_num = '0.2'
version_num = '1.0'

if version_num == '0.0':
    static_suffix = '_static_layers.h5'
    id_path = f'{DATA_ROOT}/identification'
if version_num == '0.1':
    static_suffix = '_Static.h5'
    id_path = f'identification'
if version_num == '0.2' or version_num == '1.0':
    static_suffix = f'v{version_num}.h5'
    id_path = f'identification'

# Set name of output containing all burst products for streaming
import os 

orig_validation_bursts = Path(f'validation_bursts_target_v{version_num}.csv')

validation_bursts = Path(f'getallbursts_validation_bursts_target_v{version_num}.csv')
validation_csv = Path(f'getallbursts_table_validation_bursts_target_v{version_num}.csv')

## Load function to query S3 bucket

In [3]:
def get_matching_s3_keys(bucket, prefix='', suffix='', burstId='', version_num='', avoid_bursts=[]):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix.
    :param suffix: Only fetch keys that end with this suffix.
    :param burstId: Only fetch keys that match burstId.
    :param burstId: Only fetch keys that match version num.
    """
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    kwargs = {'Bucket': bucket}

    # If the prefix is a single string (not a tuple of strings), we can
    # do the filtering directly in the S3 API.
    # Efficiently capture only matches that share relevant filename prefix
    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix + '/' + burstId

    # # set versioning string
    # version_num = '_v' + version_num + '_'

    # Max query for `list_object_v2` is only 1000
    # Need to institute while loop to circumvent
    while True:
        # 'Contents' contains information about the listed objects
        # Sort by last modified in order to get most recent static layer
        resp = s3.list_objects_v2(**kwargs)

        try:
            valid_results = resp['Contents']
        except KeyError: 
            break
            
        get_last_modified = lambda valid_results: int(valid_results['LastModified'].strftime('%s'))
        valid_results = [obj['Key'] for obj in sorted(valid_results, key=get_last_modified)]

        # filter by suffix, version number, and remove legacy products
        valid_results = [i for i in valid_results if i.endswith(suffix)]
        valid_results = [i for i in valid_results if version_num in i]
        valid_results = [i for i in valid_results if '_v0.0_202303' not in i]
        if avoid_bursts != []:
            if 'static' in prefix.lower() or 'static' in suffix.lower():
                valid_results = [i.name.split(prefix)[1][:15].replace('-', \
                                 '_').lower() \
                                 for i in valid_results \
                                 if i not in avoid_bursts]
        for key in valid_results:
            yield key

        # The S3 API is paginated, returning up to 1000 keys at a time.
        # Pass the continuation token into the next response, until we
        # reach the final page (when this field is missing).
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

    return

## Access or initiate validation table containing links to S3 bucket

In [4]:
# access dataframe from file, if it exists
if validation_csv.is_file():
    df = pd.read_csv(validation_csv)
    validation_bursts_df = gpd.GeoDataFrame(
        df.loc[:, [c for c in df.columns if c != "geometry"]],
        geometry=gpd.GeoSeries.from_wkt(df["geometry"])
        )
else:
    # otherwise, initialize dataframe
    validation_bursts_df = gpd.GeoDataFrame()
    # add placeholder columns
    validation_bursts_df['burst_id'] = None
    validation_bursts_df['date'] = None
    validation_bursts_df['cslc_url'] = None
    validation_bursts_df['cslc_static_url'] = None
    validation_bursts_df['orbit_pass_direction'] = None
    validation_bursts_df['frame_ids'] = None

## Query server and build up validation table

In [5]:
# get new bursts
burstId_list_existing = pd.read_csv(orig_validation_bursts)['burst_id'].to_list()
new_bursts = []
cslc_static_url_base = 'OPERA_L2_CSLC-S1-STATIC_'
for static_key in get_matching_s3_keys(bucket=bucket,
                                       prefix=prefix,
                                       suffix=static_suffix,
                                       burstId=cslc_static_url_base,
                                       version_num=version_num,
                                       avoid_bursts=burstId_list_existing):
    if static_key is not None:
        cslc_static_burst = Path(static_key).name
        cslc_static_burst = cslc_static_burst.split(cslc_static_url_base)
        cslc_static_burst = cslc_static_burst[1][:15]
        cslc_static_burst = cslc_static_burst.replace('-','_')
        cslc_static_burst = cslc_static_burst.lower()
        if cslc_static_burst not in new_bursts:
            new_bursts.append(cslc_static_burst)

# remove duplicates sanity check
new_bursts = list(set(new_bursts))

In [6]:
# update burst file
update_burst_df = pd.read_csv(orig_validation_bursts)

# prep list to append
append_newbursts = [(i, 'Untitled') for i in new_bursts]
df_append_newbursts = pd.DataFrame(append_newbursts, columns=['burst_id', 'cr_network'])

# append to burst file
update_burst_df = pd.concat([update_burst_df, df_append_newbursts], ignore_index=True)
#removing duplcates
update_burst_df  = update_burst_df.drop_duplicates(['burst_id', 'cr_network'])
update_burst_df = update_burst_df.reset_index(drop=True)

# save table to file
update_burst_df.to_csv(validation_bursts, index=False)

In [7]:
## Access premade, static table containing all bursts identified for CSLC Cal/Val activities

# read list of bursts used for validation
if validation_bursts.is_file():
    df = pd.read_csv(validation_bursts)
    burstId_list = df['burst_id'].to_list()
else:
    raise Exception(f'Expected burst record {validation_bursts.absolute()} '
                    'not found. Check working directory.')

In [8]:
## Access json matching bursts to frame IDs

import requests
from io import BytesIO
import zipfile
import json

# URL of the ZIP file containing the JSON file
repo_zip_url = 'https://github.com/opera-adt/burst_db/releases/download/v0.3.1/opera-s1-disp-burst-to-frame-0.3.1.json.zip'

# Download the ZIP file
response = requests.get(repo_zip_url)
zip_data = BytesIO(response.content)

# Extract the JSON file from the ZIP archive
with zipfile.ZipFile(zip_data, 'r') as zip_ref:
    # Assuming your JSON file is named 'data.json' within the ZIP
    json_data = zip_ref.read('opera-s1-disp-burst-to-frame.json')

# Load the JSON data
burst_to_frame_json = json.loads(json_data.decode('utf-8'))['data']

In [9]:
# query products on S3 bucket
burstId_list = pd.read_csv(validation_bursts)['burst_id']

for burstId in burstId_list:
    for sensor in ['S1A','S1B']:
        print(f'BurstID: {burstId}, sensor: {sensor}')

        # adjust burst strings to reflect product name convention
        query_burstId = burstId.upper().replace('_','-')

        if version_num == '0.2' or version_num == '1.0':
            cslc_url_base = 'OPERA_L2_CSLC-S1_' + query_burstId 
            suffix = sensor + '_VV_v' + version_num + '.h5'
        else:
            cslc_url_base = 'OPERA_L2_CSLC-'+sensor+'_IW_' + query_burstId + '_VV_'

        cslc_static_url = None
        
        for key in get_matching_s3_keys(bucket=bucket,
                                        prefix=prefix,
                                        suffix=suffix,
                                        burstId=cslc_url_base,
                                        version_num=version_num):
            # only proceed if file not already captured in records
            # and only if there is a valid corresponding static layer
            cslc_url = f'{s3_path}/{key}'
            cslc_static_url_base = Path(cslc_url)
            cslc_static_url_base = cslc_static_url_base.parent.name.split(f'_v{version_num}_')[0]

            # 0.1 static layers have different naming conventions
            # which do not reflect a given acquisition date
            if version_num == '0.1':
                cslc_static_url_base = 'OPERA_L2_CSLC-'+sensor+'_IW_' + query_burstId + '_VV_'
            if version_num == '0.2' or version_num == '1.0':
                cslc_static_url_base = 'OPERA_L2_CSLC-S1-STATIC_' + query_burstId 
            
            # avoid querying the same static URL for v0.1
            # for which it is the same for each burst
            if cslc_static_url == None or version_num == '0.0':
                for static_key in get_matching_s3_keys(bucket=bucket,
                                                    prefix=prefix,
                                                    suffix=static_suffix,
                                                    burstId=cslc_static_url_base,
                                                    version_num=version_num):
                    if static_key is not None:
                        cslc_static_url = f'{s3_path}/{static_key}'
                        break

            if cslc_url not in validation_bursts_df['cslc_url'].values.astype(str):
                # get date
                file_path = Path(key)
                filename = file_path.name

                if version_num == '0.2' or version_num == '1.0':
                    date = filename.split('_')[4][:8]
                else:
                    date = filename.split('_')[-3][:8]

                # check if geometry and orbit pass direction for burst already in df
                geom_check = validation_bursts_df['burst_id'] == burstId
                idx_geo = next(iter(geom_check.index[geom_check]), False)
                if idx_geo != False:
                    geometry = validation_bursts_df.loc[idx_geo]['geometry']
                    orbit_pass_direction = \
                        validation_bursts_df.loc[idx_geo]['orbit_pass_direction']
                else:
                    # otherwise, read file to access geometry
                    s3f = fsspec.open(cslc_url, mode='rb', anon=True,
                                        default_fill_cache=False)
                    with h5py.File(s3f.open(),'r') as h5:
                        geometry = h5[f'{id_path}/'
                                      'bounding_polygon'][()].astype(str)
                        orbit_pass_direction = \
                            h5['identification']['orbit_pass_direction'][()].astype(str)
                    geometry = wkt.loads(geometry)

                # create dictionary for this file
                file_dict = {'burst_id': burstId,
                             'date': date,
                             'cslc_url': cslc_url,
                             'cslc_static_url': cslc_static_url,
                             'geometry': geometry,
                             'orbit_pass_direction': orbit_pass_direction,
                             'frame_ids': burst_to_frame_json[burstId]['frame_ids']}
                # append to records
                validation_bursts_df = pd.concat([validation_bursts_df, \
                    gpd.GeoDataFrame([file_dict])], ignore_index=True)

BurstID: t004_006658_iw3, sensor: S1A


BurstID: t004_006658_iw3, sensor: S1B


BurstID: t004_006660_iw3, sensor: S1A


BurstID: t004_006660_iw3, sensor: S1B


BurstID: t034_071054_iw2, sensor: S1A


...

BurstID: t107_227882_iw1, sensor: S1B


BurstID: t004_006647_iw1, sensor: S1A


BurstID: t004_006647_iw1, sensor: S1B


BurstID: t044_093112_iw2, sensor: S1A


BurstID: t044_093112_iw2, sensor: S1B


## Save validation table

In [10]:
#removing duplcates
validation_bursts_df  = validation_bursts_df.drop_duplicates(['burst_id', 'date'])

# sort by time
validation_bursts_df = validation_bursts_df.sort_values(by=['burst_id', 'date'], ascending=[True, True]).reset_index(drop=True)

# save table to file
validation_bursts_df.to_csv(validation_csv, index=False)